<a href="https://colab.research.google.com/github/Jeffrey-Ede/Miscellaneous/blob/master/Lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# #Install auto-sklearn with build dependencies

!sudo apt-get install build-essential swig 
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install 
!pip install auto-sklearn==0.10.0

# !sudo apt-get install build-essential swig
# !pip install auto-sklearn==0.11.1

#!pip install scikit-optimize

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   207  100   207    0     0   1095      0 --:--:-- --:--:-- --:--:--  1101
  Using cached https://files.pythonhosted.org/packages/e2/4c/6111b9a325f29527d7f262e2ee8c730d354b47a728d955e186dacad57a0d/scikit_learn-0.24.1-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: auto-sklearn 0.10.0 has requirement scikit-learn<0.23,>=0.22.0, but you'll have scikit-learn 0.24.1 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Using cached https://files.

In [55]:
#Libraries that are likely to be useful
import numpy as np
import pandas as pd
import scipy

import tensorflow as tf

import os
import itertools

In [56]:
#Download data from my Google Drive
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Z84X9RLCaLyLSfATKEjgr9hGO7fm2ERk' -O acea.zip && mkdir -p ~/data/acea

--2021-01-30 15:18:33--  https://docs.google.com/uc?export=download&id=1Z84X9RLCaLyLSfATKEjgr9hGO7fm2ERk
Resolving docs.google.com (docs.google.com)... 74.125.20.100, 74.125.20.101, 74.125.20.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.20.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7afkn3j3k1pt2m4q6dl0g4mnv1scg7rc/1612019850000/14151923763028593793/*/1Z84X9RLCaLyLSfATKEjgr9hGO7fm2ERk?e=download [following]
--2021-01-30 15:18:34--  https://doc-0g-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7afkn3j3k1pt2m4q6dl0g4mnv1scg7rc/1612019850000/14151923763028593793/*/1Z84X9RLCaLyLSfATKEjgr9hGO7fm2ERk?e=download
Resolving doc-0g-a8-docs.googleusercontent.com (doc-0g-a8-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-0g-a8-docs.googleusercontent.com (doc-0g-a8-docs.g

In [57]:
#Decompress datasets
import zipfile

with zipfile.ZipFile('acea.zip', 'r') as zip_ref:
  zip_ref.extractall('acea')

#Specify dataset file
input_file = "acea/Lake_Bilancino.csv"

In [58]:
#NOTE: I'm not inputting information about seasonal variation, such as the month.
#My thought is that it's probably implicit if you use a large enough time window
#However, this probably merits further investigation

NUM_INPUT_DATA = 64 #Days of data to input to model
TRAIN_FRAC = 0.9 #Can be large if we're just investigating ability to generalize to 1-2 years

from sklearn.preprocessing import QuantileTransformer

from sklearn.model_selection import PredefinedSplit

#Load data
df = pd.read_csv(input_file, header=0)

headers = list(df.columns.values)
data = np.float32(df.values[:, 1:]) #Could probably use 16-bit

#Only select data with at least one input feature
#data = np.stack([x for x, f in zip(data, np.isfinite(data)) if np.sum(f[:6]) > 0])

#If rows that contain no input feature are removed, then there's only one data column
#with a NaN. That means that it's difficult to train the model to learn that something
#like -1 means NaN. Avoid issue by stripping all rows with NaNs
data = np.stack([x for x, f in zip(data, np.isfinite(data)) if np.sum(f[:6]) == 6])

#Quantile transforms to normalize features to [0, 1]. For general case where data
#contains missing values
qts = []
for i in range(data.shape[1]):
  #Use histogram normalization to scale features to have values in [0, 1]
  qt = QuantileTransformer(random_state=0)
  col = data[:, i:i+1]
  is_data = np.isfinite(col)

  qt.fit_transform(np.expand_dims(col[is_data], -1))
  qts.append(qt)

  #Normalize data
  col[is_data] = qt.transform(np.expand_dims(col[is_data], -1))[:, 0]

  #In general, can replace missing data with -1
  col[~is_data] = -1

  data[:, i:i+1] = col


#Dataset is tiny, so can increase its size 100 times in RAM without
#any issue
data = np.stack([data[i:-NUM_INPUT_DATA+i] for i in range(NUM_INPUT_DATA)], axis=1)

#Test ability to generalize to latest data, which is at the bottom of the dataset
#Note that this data is likely the most valuable for training to generalize beyond 
#the dataset, so another partition or validation fold may be preferable. 
train_size = int(TRAIN_FRAC*data.shape[0])

x_train = data[:train_size,:,:-2]
prev_y_train = data[:train_size,:NUM_INPUT_DATA-1,-2:]
y_train = data[:train_size,NUM_INPUT_DATA-1,-2:]
x_val = data[train_size:,:,:-2]
prev_y_val = data[train_size:,:NUM_INPUT_DATA-1,-2:]
y_val = data[train_size:,NUM_INPUT_DATA-1,-2:]

# x = data[:,:,:-2]
# y = data[:,:,-2:]

test_fold = [-1 for _ in range(train_size)] + [0 for _ in range(NUM_INPUT_DATA-train_size)]
ps = PredefinedSplit(test_fold=test_fold)

In [59]:
#There might be a large difference in errors, so easiest to develop models for each 
#prediction separately to minimize development time

from autosklearn.regression import AutoSklearnRegressor

print(x_train.shape)
print(prev_y_train.shape)

#training data was originally shaped for CNN, so need to reshape 
x_train = np.concatenate(tuple([x_train[:,i,:] for i in range(NUM_INPUT_DATA)] +
                               [prev_y_train[:,i,:] for i in range(NUM_INPUT_DATA-1)]), axis=-1)
x_val = np.concatenate(tuple([x_val[:,i,:] for i in range(NUM_INPUT_DATA)] + 
                             [prev_y_val[:,i,:] for i in range(NUM_INPUT_DATA-1)]), axis=-1)

pred_y_val = []
for i in range(2):
  automl = AutoSklearnRegressor(
    time_left_for_this_task=60*60, per_run_time_limit=5*60, n_jobs=-1
  )

  automl.fit(x_train, y_train[:,i])

  print(automl.show_models())

  pred_y_val.append(automl.predict(x_val))

pred_y_val = np.stack(pred_y_val, axis=-1)

(5364, 64, 6)
(5364, 63, 2)
[(0.460000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'select_rates_regression', 'regressor:__choice__': 'gradient_boosting', 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.7484939216574421, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.20183301399810935, 'feature_preprocessor:select_rates_regression:alpha': 0.18375456889543734, 'feature_preprocessor:select_rates_regression:mode': 'fdr', 'feature_preprocessor:select_rates_regression:score_func': 'f_regression', 'regressor:gradient_boosting:early_stop': 'off', 'regressor:gradient_bo

In [60]:
#Dataset is small enough to process validation data in a single batch

#Should probably freeze batch norm -- val set may have significantly different characterstics --
#but let's not worry about stuff like that here (removed batch norm to avoid this issue)

print(pred_y_val.shape)

for i, qt in enumerate(qts[-2:]):
  rms = np.sqrt(np.sum((
      qt.inverse_transform(pred_y_val[:,i:i+1]) - qt.inverse_transform(y_val[:,i:i+1])
  )**2))

  if i == 0:
    print("Lake_Level RMS:", rms, "liters/sec")
  elif i == 1:
    print("Flow_Rate RMS:", rms, "metres")

(597, 2)
Lake_Level RMS: 4.791936990166326 liters/sec
Flow_Rate RMS: 70.52843514565923 metres


In [61]:
# #Feature selection
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.feature_selection import RFECV

# estimator = ExtraTreesRegressor()
# estimator.fit(input_features, target_features)

# selector = RFECV(estimator, step=1, cv=5)
# selector = selector.fit(input_features, target_features[:, 1])
# print(selector.support_)
# print(estimator.feature_importances_)

In [62]:
# #Apply automatic machine learning to regression
# import autosklearn.regression
# automl = autosklearn.regression.AutoSklearnRegressor(
#     time_left_for_this_task=120,
#     per_run_time_limit=30,
#     n_jobs=1
# )
# automl.fit(
#     X_train_transformed, 
#     y_train
# )

In [63]:
# automl.sprint_statistics()

In [64]:
# automl.show_models()

In [65]:
# import sklearn.metrics
# predictions = automl.predict(X_test_transformed)
# sklearn.metrics.r2_score(y_test, predictions)

In [66]:
# #NOTE: The next two code blocks are for a 1D CNN, which yields surprisingly bad performance
# #Possibly an issue that needs debugging. Swap to AutoML after
# #Results were Lake_Level RMS: 34.957657, Flow_Rate RMS: 132.83891

# #Define a model with tunable hyperparameters for Bayesian optimization

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Flatten, Dense
# from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.optimizers.schedules import ExponentialDecay
# from tensorflow.keras.losses import MeanSquaredError

# def build_model(start_hidden=64, batch_size=64, learning_rate=1e-3, epochs=100):
#     """
#     1D CNN with fully connected layers at the end
#     """

#     #Data augmentation pipeline could be added as part of the model
#     # ...

#     padding = "VALID"

#     #Sequential model maps inputs to target outputs
#     model = Sequential()

#     #Convolutions to exploit temporal correlation
#     #Could probably improve by using dilated convolutions
#     for i in range(int(np.log2(NUM_INPUT_DATA//8))):
#       print(i)
#       if not i:
#         model.add(Conv1D(filters=max(start_hidden*2**2, 512), kernel_size=3, strides=1, 
#                          input_shape=(NUM_INPUT_DATA, x_train.shape[-1]), padding=padding))
#       else:
#         model.add(Conv1D(filters=max(start_hidden*2**2, 512), kernel_size=3, strides=1, padding=padding))

#       model.add(Conv1D(filters=max(start_hidden*2**2, 512), kernel_size=3, strides=2, padding=padding))
#       model.add(BatchNormalization())
#       model.add(Activation("relu"))

#     model.add(Flatten())

#     #Fully connect to outputs
#     for _ in range(2):
#       model.add(Dense(4*start_hidden))
#       model.add(BatchNormalization())
#       model.add(Activation("relu"))
    
#     model.add(Dense(y_train.shape[-1]))

#     #Optimization
#     # lr_schedule = ExponentialDecay(
#     #     initial_learning_rate=learning_rate,
#     #     decay_steps=NUM_INPUT_DATA, 
#     #     decay_rate=np.exp(np.log(1-end_decay)/epochs), 
#     #     staircase=True
#     # )
#     optimizer = Adam(lr=learning_rate)
#     model.compile(optimizer=optimizer, loss=MeanSquaredError())

#     return model

# # model = build_model()
# # for l in model.layers:
# #   print(l.output_shape)

# # model.fit(x, y)

In [67]:
# #from skopt import BayesSearchCV #Unfortunately, this is causing a lot of bugs
# from sklearn.model_selection import GridSearchCV
# from skopt.space import Real, Categorical, Integer

# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from tensorflow.keras.callbacks import EarlyStopping

# from tensorflow.keras.callbacks import LearningRateScheduler


# #Trivial search space for (likely) most important hyperparameter
# search_space = {
#     "learning_rate": [0.01]#(0.01, 0.003, 0.001, 0.0003, 0.0001)
# }

# #Prepare model for interfacing with sklearn. It's a thin wrapper
# keras_model = KerasRegressor(build_model)

# #Simple grid search. Could use RandomizedSearchCV of BayesSearchCV for higher-dimensional search space
# #with 5-fold cross-validation
# search = GridSearchCV(keras_model, search_space, n_jobs=-1, cv=5, verbose=10)

# scheduler = lambda epoch, lr: 0.97*lr #Vaguely sensible stepwise decay rate
# lr_callback = LearningRateScheduler(scheduler)

# search = search.fit(
#     x_train, 
#     y_train,
#     epochs=200, #Probably far higher than needed.
#     validation_data=(x_val, y_val),
#     callbacks=[lr_callback], #EarlyStopping(patience=10),Stop if no improvement for 10 epochs
#     verbose=2
# )

# print(search.best_params_)
# print(search.best_score_)
# print(search.best_estimator_)

# pred_y_val = search.best_estimator_.predict(x_val)